In [14]:
import torch
import torch.nn as nn

Check quantileloss functions

In [15]:
def LoopLoss(preds, target, quantiles):
    def _tilted_loss(q, e):
        return torch.max((q-1) * e, q * e).unsqueeze(1)

    err = target - preds
    losses = [_tilted_loss(q, err[:, i])  # calculate per quantile
              for i, q in enumerate(quantiles)]

    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss


class VectorLoss(nn.Module):
    """
    Calculates the quantile loss function.

    Attributes
    ----------
    self.pred : torch.tensor
        Predictions.
    self.target : torch.tensor
        Target to predict.
    self.quantiles : torch.tensort
    """
    def __init__(self, quantiles):
        super(VectorLoss,self).__init__()
        self.pred = None
        self.targes = None
        self.quantiles = quantiles
        
    def forward(self,pred ,target):
        """
        Computes the loss for the given prediction.
        """
        error = target - pred
        upper =  self.quantiles * error
        lower = (self.quantiles - 1) * error 

        losses = torch.max(lower, upper)
        loss = torch.mean(torch.sum(losses, dim=1))
        
        return loss

In [7]:
quantiles = torch.linspace(0, 1, 9)
quantiles

tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750, 1.0000])

In [11]:
outputs = torch.randn((300, 9))
target = torch.randn((300, 1))

In [16]:
criterion = VectorLoss(quantiles)
criterion(outputs, target)

tensor(5.1973)

In [17]:
LoopLoss(outputs, target, quantiles)

tensor(5.1973)